# Statistics in Python

There are two major modules for doing statistical analyses in Python:

* [Scipy](http://docs.scipy.org/doc/scipy/reference/stats.html) - basic statistics and distribution fitting
* [Statsmodels](http://statsmodels.sourceforge.net/stable/index.html) - advanced statistical modeling focused on linear models
(including ANOVA, multiple regression, generalized linear models, etc.)

To see the full functionality of these modules you'll need to look through
their pages, but here are a few examples to show you that a lot of the
standard statistical tests and models you need to perform can be easily
done using Python.

Imports
-------
You'll want the `stats` module from Scipy and the `api` and `formula.api`
modulesfrom statsmodels. We'll also go ahead and import Numpy for use in
generating data and Matplotlib for some graphing.

In [ ]:
!pip install scipy
!pip install statsmodels
!pip install numpy
!pip install matplotlib
!pip install pandas

In [ ]:
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

## Descriptive Statistics

`stats.describe()` gives basic descriptive statistics on any list of numbers. It returns (in the following order) the size of the data, it's min and max, the mean, the variance, skewness, and kurtosis.

In [ ]:
x = [1, 2, 3, 4, 5]
stats.describe(x)

We can also get this kind of information for columns in a DataFrame using the `.describe()` method.

In [ ]:
data = pd.DataFrame([[1, 2, 3.5], [2, 2.4, 3.1], [3, 1.8, 2.5]], columns=['a', 'b', 'c'])
print (data)
data.describe()

## T-tests

Standard 1-sample and 2-sample t-tests (as well as many other basic statistical tests) are available in `scipy.stats`.
T-tests return two values, the t-statistic and the p-value. First, let's generate some data that is normally distributed around zero.

In [ ]:
x1 = np.random.randn(100, 1)
x2 = np.random.randn(100, 1)

### One-sample t-test

To determine if the mean of `x1` is different from some number use a one-sample t-test.
We'll compare the mean of `x1` to both 0 (which it shouldn't be different from) and to 1 (which it should be different from).

In [ ]:
tstat, pval = stats.ttest_1samp(x1, 0)
print ("Comparison of the mean of x1 to 0.\nT-statistic = %s; P-value = %s." % (tstat, pval))

tstat, pval = stats.ttest_1samp(x1, 1)
print ("Comparison of the mean of x1 to 5.\nT-statistic = %s; P-value = %s." % (tstat, pval))

### Two-sample t-test

To determine if the mean of two different sets of numbers are different from one another wuse a two-sample t-test. We'll compare the means of `x1` and `x2`, which should be different from one another since they should both be about zero.

In [ ]:
tstat, pval = stats.ttest_ind(x1, x2)
print ("Comparison of the means of x1 and x2.\nT-statistic = %s; P-value = %s." % (tstat, pval))

## Distribution fitting and analysis

`scipy.stats` also includes a powerful general system for working with statistical distribution.
Let's generate some random normally distributed numbers to analyze.

In [ ]:
u, sigma = 4, 2
random_numbers = stats.norm.rvs(u, sigma, size=50)
random_numbers

We can then fit distributions to this data.

In [ ]:
stats.norm.fit(random_numbers)

## Regression

You can do simple univariate OLS regression in Scipy,
but for anything more complicated you'll need statsmodels,
so we'll just do the basics in statsmodels as well.

First, we'll generate some data and plot it.

In [ ]:
#generate some data
x = np.array(range(20))
y = 3 + 0.5 * x + 2 * np.random.randn(20)
data = pd.DataFrame(list(zip(x, y)), columns=['x', 'y'])

#plot the data
plt.plot(data['x'], data['y'], 'bo')
plt.show()

Now let's do a regression. We'll use formulas to specify regression models.
This allows us to write our code in simple and intuitive ways,
and means the we don't have to remember how to create design matrices for more complicated models.
To do this we'll need to be using Pandas to manage the data.
The regression function is name `ols` for "ordinary least-squares" the standard approach to regression.
It takes two arguments. The first is a formula describing the regression we want to fit.
In this case we just want to model the effect of `x` on `y` so we use `y ~ x`,
where `x` and `y` are the names of columns in a data frame.
The second argument is the name of the data frame that contains the columns `x` and `y`.

In [ ]:
results = smf.ols('y ~ x', data).fit()
print (results.summary())

So, ``.summary()`` presents us with most of the information we would want about the regression.
We can also pull this information out in individual pieces. For example,

In [ ]:
intercept, slope = results.params
r2 = results.rsquared
print (slope, intercept, r2)

This makes it easy to store the key results of large numbers of analyses, or present
the results in alternative ways, like graphs.

In [ ]:
plt.plot(data['x'], data['y'], 'bo')
x = np.array([min(x), max(x)])
y = intercept + slope * x
plt.plot(x, y, 'r-')
plt.show()

You'll notice that in order to plot the regression line what we actually do is
plot a line with the appropriate slope and intercept by:

1. taking the minimum and maximum values of x
2. calculating their values of y based on the regression results
3. and plotting those two points with a straight line connecting them and no symbols

### Multiple-regression

Multiple-regression works the same way, but with additional terms in the formula.

In [ ]:
import pandas as pd

#generate some data
x = np.random.randn(50)
z = np.random.randn(50)
noise = np.random.randn(50)
y = 3 + 0.5 * x + 1.5 * z + noise

data = pd.DataFrame(list(zip(x, y, z)), columns=['x', 'y', 'z'])
results = smf.ols('y ~ x + z', data).fit()
print (results.summary())

This makes it easy to do more complicated designs, including interactions.

In [ ]:
results = smf.ols('y ~ x + z + x*z', data).fit()
print (results.summary())

We can also include transformations in formulas using functions from Numpy.

In [ ]:
results = smf.ols('y ~ x + np.log10(z)', data).fit()
print (results.summary())

### ANOVA
Using formulas also makes it easy to conduct statistical tests that are based on linear models.
ANOVA is simply a linear model with appropriate dummy variables set up for each factor.
To do this in statsmodels we simply use ``C()`` to tell the module that the
variable of interest is categorical.

This time, let's start by grabbing some data from the web.

In [ ]:
url = 'http://stats191.stanford.edu/data/rehab.csv'
rehab_table = pd.read_table(url, delimiter=",")
rehab_table

To see if the time to rehabilitate an injury is related to the fitness category we do the same as above,
but wrapping the predictor in ``C()``.

In [ ]:
results = smf.ols('Time ~ C(Fitness)', rehab_table).fit()
print (results.summary())

While all of the information that we want is technically present in this table,
we typically want it presented in more standard fashion for ANOVA.
We can do this using the ``anova_lm`` function.

In [ ]:
from statsmodels.stats.anova import anova_lm

anova_lm(results)

### And lots more

Statsmodels includes much more advanced functionality including:

* Generalized Least Squares (i.e., correlated error structures such as for spatial and comparative analysis)
* Generalized Linear Models (i.e., non-normal error)
* Robust Linear Models
* Regression with Discrete Dependent Variable (e.g., logistic regression)
* Time Series analysis